# Deep Learning
# DL04 Entrenamiento Pytorch-Solucion



## <font color='blue'>Entrenando las redes neuronales</font>

#### No ejecute el notebook completo. También tenga cuidado con el numero de epochs. El proceso puede ser lento.

### Función Loss o de perdida.

La red que construimos en la parte 2, no sabe nada sobre nuestros dígitos escritos a mano. Sin embargo para que Las redes neuronales con activaciones no lineales funcionan como aproximadores de funciones universales, se debe enseñar la forma de la función. Que ocurre con las imagenes?, como le asignamos una función de aproximación o de aprendizje?. Las imágenes de dígitos escritos a mano la podemos asociar a probabilidades de clase. El poder de las redes neuronales es que podemos entrenarlas para aproximar esta función, y básicamente cualquier función con suficientes datos y tiempo de cálculo.

![Log](https://drive.google.com/uc?export=view&id=1d7hNBU9q8x7D8XQPqDg1xqXuxrC9hyjS)


Al principio, la red es ingenua, no conoce la función que asigna las entradas a las salidas. Entrenamos la red mostrándole ejemplos de datos reales, luego ajustamos los parámetros de la red de modo que se aproxime a esta función.

Para encontrar estos parámetros, necesitamos saber qué tan mal está prediciendo la red los resultados reales. Para esto calculamos una **función de pérdida (loss function)** (también llamada __función de costo__), una medida de nuestro error de predicción. Dependiendo de qué función decosto usemos dependerá, a posteriori, el proceso de __back propagation__. Por ejemplo, la pérdida cuadrática media a menudo se usa en problemas de regresión y clasificación binaria.
<br>
$$
\large \ell = \frac{1}{2n}\sum_i^n{\left(y_i - \hat{y}_i\right)^2}
$$
<br>
Donde $n$ es el numero de ejemplo de entrenamiento, $y_i$ son los valores reales o etiquetas, y $\hat{y}_i$ son los valores predecidos.

### Optimización de los pesos




Al minimizar esta pérdida con respecto a los parámetros de la red, podemos encontrar configuraciones donde la pérdida es mínima y la red puede predecir las etiquetas correctas con alta precisión. Encontramos este mínimo usando un proceso llamado **descenso de gradiente**. El gradiente es la pendiente de la función de pérdida y apunta en la dirección del cambio más rápido. Para llegar al mínimo en la menor cantidad de tiempo, entonces queremos seguir el gradiente (hacia abajo). Puedes pensar en esto como descender una montaña siguiendo la pendiente más empinada hasta la base.


![Log](https://drive.google.com/uc?export=view&id=1-kPnRcfH8bhPH1vzQsWrRGC_0Nptc7jR)


### Backpropagation

Para redes de una sola capa, el descenso de gradiente es fácil de implementar. Sin embargo, es más complicado para redes neuronales multicapa más profundas como la que hemos construido. Lo suficientemente complicado como para que pasaron unos 30 años antes de que los investigadores descubrieran cómo entrenar redes multicapa.

El entrenamiento de redes multicapa se realiza a través de **backpropagation**, que en realidad es solo una aplicación de la regla de la cadena del cálculo. Es más fácil de entender si convertimos una red de dos capas en una representación gráfica.


![Log](https://drive.google.com/uc?export=view&id=1I7-fkRYztOZDU1P1JTo17DCFMc6O8tIO)


En el paso directo a través de la red, nuestros datos y operaciones van de abajo hacia arriba aquí. Pasamos la entrada $x$ a través de una transformación lineal $L_1$ con pesos  $W_1$ y sesgos $b_1$. La salida luego pasa por la operación sigmoidea $S$ y otra transformación lineal $L_2$. Finalmente calculamos la pérdida $ \ell $. Usamos la pérdida como una medida de cuán malas son las predicciones de la red. El objetivo entonces es ajustar los pesos y sesgos para minimizar la pérdida.

Para entrenar los pesos con descenso de gradiente, propagamos el gradiente de la pérdida hacia atrás a través de la red. Cada operación tiene algún gradiente entre las entradas y salidas. A medida que enviamos los gradientes hacia atrás, multiplicamos el gradiente entrante con el gradiente para la operación. Matemáticamente, esto es solo calcular el gradiente de la pérdida con respecto a los pesos usando la regla de la cadena.


$$
\large \frac{\partial \ell}{\partial W_1} = \frac{\partial L_1}{\partial W_1} \frac{\partial S}{\partial L_1} \frac{\partial L_2}{\partial S} \frac{\partial \ell}{\partial L_2}
$$


Actualizamos nuestros pesos usando este gradiente con cierta tasa de aprendizaje $ \ alpha $.

$$
\large W^\prime_1 = W_1 - \alpha \frac{\partial \ell}{\partial W_1}
$$

La tasa de aprendizaje $ \ alpha $ se establece de manera que los pasos de actualización de peso sean lo suficientemente pequeños como para que el método iterativo se establezca en un mínimo.

### Losses in PyTorch

Comencemos por ver cómo calculamos el loss con PyTorch. A través del módulo `nn`, PyTorch proporciona diferentes tipos de loss como por ejemplo ` nn.CrossEntropyLoss`.  En un problema de clasificación como MNIST, estamos utilizando la función softmax para predecir las probabilidades de clase. Con una salida softmax, desea utilizar la entropía cruzada como loss. Para calcular realmente el loss, primero define el criterio y luego pasa la salida de su red y las etiquetas correctas.


Para mas información mirar [the documentation for `nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss),

> Este criterio combina `nn.LogSoftmax()` y `nn.NLLLoss()` en una sola clase.
>


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms

# Normalizamos y transformamos.


transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
# Bajamos y cargamos los datos.
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



#### Un primer ejemplo utlizando CrossEntropyLoss como loss

In [ ]:
# Contruimos una red  feed-forward.
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10))

# Definiendo el loss. Recordar que esta integrado con el softmax en el caso del CrossEntropyLoss.
criterion = nn.CrossEntropyLoss()

# Obtengamos la data.
images, labels = next(iter(trainloader))
# Convirtiendo a vector.
images = images.view(images.shape[0], -1)

# Forward pass, obteniendo el logits.
logits = model(images)
# Calculando el loss
loss = criterion(logits, labels)

print(loss)

tensor(2.3037, grad_fn=<NllLossBackward0>)


#### Un segundoejemplo utlizando NLLLoss como loss

https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html

In [ ]:
# Contruimos una red  feed-forward.
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

# Definiendo el loss.
criterion = nn.NLLLoss()

# Obtengamos la data.
images, labels = next(iter(trainloader))
# Convirtiendo a vector.
images = images.view(images.shape[0], -1)

# Forward pass, obteniendo las probabilidades.
logps = model(images)
# Calculando el loss
loss = criterion(logps, labels)

print(loss)

tensor(2.3120, grad_fn=<NllLossBackward0>)


### Autograd


Ahora que sabemos cómo calcular un loss, ¿cómo lo usamos para realizar la propagación hacia atrás? Torch proporciona un módulo, 'autograd', para calcular automáticamente los gradientes de los tensores. Podemos usarlo para calcular los gradientes de todos nuestros parámetros con respecto a la pérdida. Autograd funciona haciendo un seguimiento de las operaciones realizadas en los tensores, luego retrocediendo a través de esas operaciones, calculando los gradientes en el camino. Para asegurarse de que PyTorch realiza un seguimiento de las operaciones en un tensor y calcula los gradientes, debe establecer `require_grad = True` en un tensor. Puede hacer esto en la creación con la palabra clave `require_grad`, o en cualquier momento con `x.requires_grad_ (True)`.

Puede desactivar gradientes para un bloque de código con el `torch.no_grad()`:
```python
x = torch.zeros(1, requires_grad=True)
>>> with torch.no_grad():
...     y = x * 2
>>> y.requires_grad
False
```
Tambien podemos apagar el calculo de gradientes con `torch.set_grad_enabled(True|False)`.



Los gradientes son caculados respecto de alguna variable `z` con `z.backward()`.

#### Un ejemplo concreto con calculos de gradientes.

In [ ]:
x = torch.randn(2,2, requires_grad=True)
print(x)

tensor([[-0.5177,  1.1694],
        [ 0.7481,  1.1501]], requires_grad=True)


In [ ]:
y = x**2
print(y)

tensor([[0.2681, 1.3674],
        [0.5596, 1.3226]], grad_fn=<PowBackward0>)


A continuación podemos ver la operación que creó `y`, una operación de potencia`PowBackward0`.

In [ ]:
## grad_fn muestra la funcion generada por esta variable.
print(y.grad_fn)

El módulo de autograd realiza un seguimiento de estas operaciones y sabe cómo calcular el gradiente de cada una. De esta manera, puede calcular los gradientes para una cadena de operaciones, con respecto a cualquier tensor. Reduzcamos el tensor `y` a un valor escalar, la media.

In [ ]:
z = y.mean()
print(z)

tensor(0.8794, grad_fn=<MeanBackward0>)


Puede verificar los gradientes para `x` e` y` pero actualmente están vacíos.

In [ ]:
print(x.grad)

None


Para calcular los gradientes, debe ejecutar el método `.backward` en una Variable,` z` por ejemplo. Esto calculará el gradiente para `z` con respecto a` x`

$$
\frac{\partial z}{\partial x_j} = \frac{\partial}{\partial x_j}\left[\frac{1}{4}\sum_i^n x_i^2\right] = \frac{x}{2}
$$

In [ ]:
z.backward()
print(x.grad)
print(x/2)

tensor([[-0.2589,  0.5847],
        [ 0.3740,  0.5750]])
tensor([[-0.2589,  0.5847],
        [ 0.3740,  0.5750]], grad_fn=<DivBackward0>)


### Loss and Autograd juntos

When we create a network with PyTorch, all of the parameters are initialized with `requires_grad = True`. This means that when we calculate the loss and call `loss.backward()`, the gradients for the parameters are calculated. These gradients are used to update the weights with gradient descent. Below you can see an example of calculating the gradients using a backwards pass.

Cuando creamos una red con PyTorch, todos los parámetros se inicializan con `require_grad = True`. Esto significa que cuando calculamos el loss y llamamos `loss.backward ()`, se calculan los gradientes para los parámetros. Estos gradientes se utilizan para actualizar los pesos con descenso de gradiente.

In [ ]:
# Construimos una red  feed-forward.
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(64, 10),
                      nn.Dropout(0.2),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

logps = model(images)
loss = criterion(logps, labels)

In [ ]:
print('Before backward pass: \n', model[0].weight.grad)

loss.backward()

print('After backward pass: \n', model[0].weight.grad)

Before backward pass: 
 None
After backward pass: 
 tensor([[ 0.0001,  0.0001,  0.0001,  ...,  0.0001,  0.0001,  0.0001],
        [-0.0015, -0.0015, -0.0015,  ..., -0.0015, -0.0015, -0.0015],
        [ 0.0003,  0.0003,  0.0003,  ...,  0.0003,  0.0003,  0.0003],
        ...,
        [-0.0019, -0.0019, -0.0019,  ..., -0.0019, -0.0019, -0.0019],
        [ 0.0018,  0.0018,  0.0018,  ...,  0.0018,  0.0018,  0.0018],
        [-0.0032, -0.0032, -0.0032,  ..., -0.0032, -0.0032, -0.0032]])


### Entrenemos la red.

Hay una última pieza que necesitamos para comenzar a entrenar, un optimizador que usaremos para actualizar los pesos con los gradientes. Los obtenemos del paquete [`optim` package](https://pytorch.org/docs/stable/optim.html). Por ejemplo, podemos usar el descenso de gradiente estocástico con `optim.SGD`. Se puede ver cómo definir un optimizador a continuación.

In [ ]:
from torch import optim

# Los optimizadores requieren los parámetros para optimizar y una tasa de aprendizaje
# SGD Stochactic Gradien Descent - Utiliza batchs de datos para entrenar en cada epoch
optimizer = optim.SGD(model.parameters(), lr=0.01)

## <font color='green'>**Actividad 1**</font>

Ahora sabemos cómo usar todas las partes individuales, así que es momento de ver cómo funcionan juntas.  El proceso general con PyTorch:

* Hacer un _forwar_d a través de la red
* Use la salida de red para calcular el _loss_
* Realice un paso hacia atrás a través de la red con `loss.backward ()` para calcular los gradientes
* Da un paso con el optimizador para actualizar los pesos

### Entrenamiento.

Ahora pondremos este algoritmo en un ciclo para que podamos ver todas las imágenes. Alguna nomenclatura, una pasada a través de todo el conjunto de datos se llama **epoch**. Así que aquí vamos a recorrer el `trainloader` para obtener nuestros lotes de entrenamiento. Para cada lote, haremos un pase de entrenamiento donde calcularemos la pérdida, haremos un pase hacia atrás y actualizaremos los pesos.

### Tenga cuidado con el numero de epochs. El proceso puede ser lento.

**Ayuda**

1. Defina una red por ejemplo:



```
model = nn.Sequential(***)
```
2. Defina el loss
3. Defina el optimizador
4. Itere respecto de las epochs:



```
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
      # Genere un vector de 784 caracteristicas.
      images = images.view(images.shape[0], -1)
      #Realice el forward
      # Calcule el loss.
      # Realice el backward()
      # Optimice
      # Acumule el loss.
```




<font color='green'>**Fin Actividad 1**</font>